## Ens

In [9]:
! sh ens.sh

+ OUT_EXT=.out.tmp
++ ls 22222_outs.txt 'results&test=Ind_test.txt.cleaned&time=11_30_09_33.txt&notes=ckpt-1500.txt' 'results&test=Ind_test.txt.cleaned&time=11_30_09_42.txt&notes=ckpt-2000.txt' 'results&test=Ind_test.txt.cleaned&time=11_30_09_51.txt&notes=ckpt-4000.txt'
+ TXTS='22222_outs.txt
results&test=Ind_test.txt.cleaned&time=11_30_09_33.txt&notes=ckpt-1500.txt
results&test=Ind_test.txt.cleaned&time=11_30_09_42.txt&notes=ckpt-2000.txt
results&test=Ind_test.txt.cleaned&time=11_30_09_51.txt&notes=ckpt-4000.txt'
+ for f in $TXTS
+ dos2unix 22222_outs.txt
dos2unix: converting file 22222_outs.txt to Unix format...
+ OUT=22222_outs.txt.out.tmp
+ echo 22222_outs.txt
+ echo -e '\n'
+ awk -F '	' '{print $1}' 22222_outs.txt
+ for f in $TXTS
+ dos2unix 'results&test=Ind_test.txt.cleaned&time=11_30_09_33.txt&notes=ckpt-1500.txt'
dos2unix: converting file results&test=Ind_test.txt.cleaned&time=11_30_09_33.txt&notes=ckpt-1500.txt to Unix format...
+ OUT='results&test=Ind_test.txt.cleaned&time=1

## Read ens_out

In [10]:
ens_out_file = "ens_out"
voting_file = f"voting.txt"
weighted_voting_file = f"weighted_voting.txt"

In [11]:
with open(ens_out_file) as f:
    lines = f.readlines()
    
# accs = [float(item.split("=")[1].replace(".txt", "")) for item in lines[0].split("\t")]
# print(accs)

lines = lines[3:]

In [12]:
from collections import Counter

## Voting

In [13]:
predictions = []
for line in lines:
    if line.strip() == "":
        predictions.append("\n")
        continue
    
    line_predictions = line.strip().split("\t")
    line_prediction = Counter(line_predictions).most_common()[0][0]
    predictions.append(line_prediction)

In [14]:
with open(voting_file, "w") as f:
    for prediction in predictions:
        f.write(f"{prediction}")
        if prediction != "\n":
            f.write("\n")

## Weighted-voting

In [57]:
norm_accs = [acc / sum(accs) for acc in accs]
print(norm_accs)

weighted_predictions = []
for line in lines:
    if line.strip() == "":
        weighted_predictions.append("\n")
        continue
    
    line_predictions = line.strip().split("\t")
    line_predictions_dict = {}
    for i in range(len(line_predictions)):
        line_prediction = line_predictions[i]
        norm_acc = norm_accs[i]
        
        if line_prediction not in line_predictions_dict.keys():
            line_predictions_dict[line_prediction] = norm_acc
        else:
            line_predictions_dict[line_prediction] += norm_acc
            
    line_prediction = sorted(list(line_predictions_dict.items()), key=lambda item: item[1], reverse=True)[0][0]
    weighted_predictions.append(line_prediction)

[0.12454002322620364, 0.1242042226917211, 0.12231534468525695, 0.12476389024919197, 0.12522561598410545, 0.12624700927648977, 0.12640091785479424, 0.12630297603223684]


In [58]:
with open(weighted_voting_file, "w") as f:
    for weighted_prediction in weighted_predictions:
        f.write(f"{weighted_prediction}")
        if weighted_prediction != "\n":
            f.write("\n")

## Cmp

In [64]:
! paste "Lao_train.tsv.test" voting.txt > voting_srcs_trgs_outs.txt
! paste "Lao_train.tsv.test" weighted_voting.txt > weighted_voting_srcs_trgs_outs.txt

! awk '{printf "%s\t%s\n", $2, $3}' voting_srcs_trgs_outs.txt  > voting_trgs_outs.txt
! awk '{printf "%s\t%s\n", $2, $3}' weighted_voting_srcs_trgs_outs.txt  > weighted_voting_trgs_outs.txt

! rm voting_srcs_trgs_outs.txt
! rm weighted_voting_srcs_trgs_outs.txt

In [65]:
def get_acc(trgs, outs):
    assert len(trgs) == len(outs)
    
    right_count = 0
    count = 0
    for trg, out in zip(trgs, outs):
        if trg == out:
            right_count += 1
        count += 1
        
    return right_count / count

In [66]:
def get_pairs(file):
    with open(file) as f:
        lines = f.readlines()
    
    trgs = []
    outs = []
    for line in lines:
        if line.strip() == "":
            continue
        
        trg, out = line.strip().split("\t")
        trgs.append(trg)
        outs.append(out)
    
    return trgs, outs

In [67]:
print(get_acc(*get_pairs("voting_trgs_outs.txt")))

0.902895208813733


In [68]:
print(get_acc(*get_pairs("weighted_voting_trgs_outs.txt")))

0.9046887009992314
